# LIS With MobileViTs

## 1. Our MobileViTs

In [9]:
import lightning as pl
import torch
import torch.nn as nn
import torch.functional as F
import wandb
from einops import rearrange
import torchvision
from phoenix_datasets import PhoenixVideoTextDataset
from torch.utils.data import DataLoader
import os
import gc
import numpy as np
from torchmetrics.text import WordErrorRate

In [10]:
# Optimize this impl:
# https://github.com/chinhsuanwu/mobilevit-pytorch/blob/master/mobilevit.py


class Conv2DBlock(nn.Module):

    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size,
        stride,
        padding,
        groups=1,
        bias=True,
        norm=True,
        activation=True,
    ):
        """__init__ Constructor for Conv2DBlock

        Parameters
        ----------
        in_channels : int
            Number of input channels
        out_channels : int
            Number of output channels
        kernel_size : int
            Size of the kernel
        stride : int
            Stride of the convolutional layer
        padding : int
            Padding of the convolutional layer
        groups : int
            Number of groups
        bias : bool
            Whether to use bias
        """

        super(Conv2DBlock, self).__init__()

        self.block = nn.Sequential()

        self.block.add_module(
            "conv2d",
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
                groups=groups,
                bias=bias,
            ),
        )

        if norm:
            self.block.add_module("norm", nn.BatchNorm2d(out_channels))

        if activation:
            self.block.add_module("activation", nn.SiLU())  # sigmoid(x) * x // SWISH

        self.block = self.block

    def forward(self, x):
        return self.block(x)


class MobileBlockV2(nn.Module):

    def __init__(self, in_channels, out_channels, stride, expand_ratio):
        """__init__ Constructor for MobileBlockV2


        Parameters
        ----------
        in_channels : int
            Number of input channels
        out_channels : int
            Number of output channels
        stride : int
            Stride of the convolutional layer
        expand_ratio : int
            Expansion ratio of the block
        """

        super(MobileBlockV2, self).__init__()

        assert stride in [1, 2], "Stride must be either 1 or 2"

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.stride = stride
        self.expand_ratio = expand_ratio
        self.hidden_dim = int(round(in_channels * expand_ratio))

        self.mbv2 = nn.Sequential()
        self.uses_inverse_residual = (
            self.in_channels == self.out_channels and self.stride == 1
        )

        if self.expand_ratio == 1:
            self.mbv2.add_module(
                "depthwise_3x3",
                Conv2DBlock(  # Depthwise Convolution
                    in_channels=self.hidden_dim,
                    out_channels=self.hidden_dim,
                    kernel_size=3,
                    stride=stride,
                    padding=1,
                    groups=self.hidden_dim,
                    bias=False,
                    norm=True,
                    activation=True,
                ),
            )
            self.mbv2.add_module(
                "pointwise-linear_1x1",
                Conv2DBlock(  # Pointwise-Linear Convolution
                    in_channels=self.hidden_dim,
                    out_channels=self.hidden_dim,
                    kernel_size=1,
                    stride=1,
                    padding=0,
                    groups=self.hidden_dim,
                    bias=False,
                    norm=True,
                    activation=False,
                ),
            )
        else:
            self.mbv2.add_module(
                "pointwise_1x1",
                Conv2DBlock(  # Pointwise-Linear Convolution
                    in_channels=in_channels,
                    out_channels=self.hidden_dim,
                    kernel_size=1,
                    stride=1,
                    padding=0,
                    groups=1,
                    bias=False,
                    norm=True,
                    activation=True,
                ),
            )
            self.mbv2.add_module(
                "depthwise_3x3",
                Conv2DBlock(  # Depthwise Convolution
                    in_channels=self.hidden_dim,
                    out_channels=self.hidden_dim,
                    kernel_size=3,
                    stride=stride,
                    padding=1,
                    groups=self.hidden_dim,
                    bias=False,
                    norm=True,
                    activation=True,
                ),
            )
            self.mbv2.add_module(
                "pointwise-linear_1x1",
                Conv2DBlock(  # Pointwise-Linear Convolution
                    in_channels=self.hidden_dim,
                    out_channels=self.out_channels,
                    kernel_size=1,
                    stride=1,
                    padding=0,
                    groups=1,
                    bias=False,
                    norm=True,
                    activation=False,
                ),
            )

        self.mbv2 = self.mbv2

    def forward(self, x):
        if self.uses_inverse_residual:
            return x + self.mbv2(x)
        else:
            return self.mbv2(x)


class MobileViTBlock(nn.Module):

    def __init__(
        self, hidden_dim, depth, channels, kernel_size, patch_size, mlp_dim, dropout=0.0
    ):

        super(MobileViTBlock, self).__init__()

        self.hidden_dim = hidden_dim
        self.depth = depth
        self.channels = channels
        self.kernel_size = kernel_size
        self.patch_size = patch_size
        self.mlp_dim = mlp_dim
        self.dropout = dropout

        self.local_conv = nn.Sequential(
            Conv2DBlock(
                in_channels=channels,
                out_channels=channels,
                kernel_size=kernel_size,
                stride=1,
                padding=1,
                norm=True,
                activation=True,
                bias=False,
            ),
            Conv2DBlock(
                in_channels=channels,
                out_channels=hidden_dim,
                kernel_size=1,
                stride=1,
                padding=0,
                norm=True,
                activation=True,
                bias=False,
            ),
        )

        # def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout=0.):
        self.global_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=hidden_dim,
                nhead=8,
                dim_feedforward=mlp_dim,
                dropout=dropout,
                batch_first=True,
                activation="gelu",
            ),
            num_layers=depth,
            norm=nn.LayerNorm(hidden_dim),
        )

        self.fusion_conv_preres = Conv2DBlock(
            in_channels=hidden_dim,
            out_channels=channels,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False,
            norm=True,
            activation=True,
        )

        self.fusion_conv_postres = Conv2DBlock(
            in_channels=2 * channels,
            out_channels=channels,
            kernel_size=kernel_size,
            stride=1,
            padding=1,
            bias=False,
            norm=True,
            activation=True,
        )

    def forward(self, x):

        B, T, C, H, W = x.shape
        x = rearrange(x, "b t c h w -> (b t) c h w")
        x_res = x.clone()

        # local_repr
        x = self.local_conv(x)

        ph, pw = self.patch_size

        # global_repr
        _, _, h, w = x.shape
        x = rearrange(  # reshape the image into patches for ViT input
            x,
            "(b t) d (h ph) (w pw) -> (b h w) (t ph pw) d",
            ph=ph,
            pw=pw,
            b=B,
            t=T,
        )
        x = self.global_transformer(x)
        x = rearrange(
            x,
            "(b h w) (t ph pw) d -> (b t) d (h ph) (w pw)",
            h=h // ph,
            w=w // pw,
            ph=ph,
            pw=pw,
            b=B,
            t=T,
        )

        # fusion
        x = self.fusion_conv_preres(x)
        x = torch.cat([x, x_res], dim=1)
        x = self.fusion_conv_postres(x)

        x = rearrange(x, "(b t) c h w -> b t c h w", b=B, t=T)

        return x


class MobileViT(pl.LightningModule):

    def __init__(
        self,
        dims,
        conv_channels,
        num_classes,
        vocabulary,
        expand_ratio=4,
        patch_size=(2, 2),
    ):
        super(MobileViT, self).__init__()

        self.dims = dims
        self.conv_channels = conv_channels
        self.num_classes = num_classes
        self.expand_ratio = expand_ratio
        self.patch_size = patch_size
        self.kernel_size = 3

        L = [2, 4, 3]

        self.in_conv = Conv2DBlock(
            in_channels=3,
            out_channels=conv_channels[0],
            kernel_size=self.kernel_size,
            stride=2,
            padding=1,
            norm=True,
            activation=True,
            bias=False,
        )

        self.mv2_blocks = nn.ModuleList([])

        self.mv2_blocks.append(
            MobileBlockV2(
                conv_channels[0], conv_channels[1], 1, expand_ratio=expand_ratio
            )
        )
        self.mv2_blocks.append(
            MobileBlockV2(
                conv_channels[1], conv_channels[2], 2, expand_ratio=expand_ratio
            )
        )
        self.mv2_blocks.append(
            MobileBlockV2(
                conv_channels[2], conv_channels[3], 1, expand_ratio=expand_ratio
            )
        )
        self.mv2_blocks.append(
            MobileBlockV2(
                conv_channels[2], conv_channels[3], 1, expand_ratio=expand_ratio
            )
        )
        self.mv2_blocks.append(
            MobileBlockV2(
                conv_channels[3], conv_channels[4], 2, expand_ratio=expand_ratio
            )
        )
        self.mv2_blocks.append(
            MobileBlockV2(
                conv_channels[5], conv_channels[6], 2, expand_ratio=expand_ratio
            )
        )
        self.mv2_blocks.append(
            MobileBlockV2(
                conv_channels[7], conv_channels[8], 2, expand_ratio=expand_ratio
            )
        )

        self.mvit_blocks = nn.ModuleList([])

        self.mvit_blocks.append(
            MobileViTBlock(
                dims[0],
                L[0],
                conv_channels[5],
                self.kernel_size,
                patch_size,
                int(dims[0] * 2),
            )
        )
        self.mvit_blocks.append(
            MobileViTBlock(
                dims[1],
                L[1],
                conv_channels[7],
                self.kernel_size,
                patch_size,
                int(dims[1] * 4),
            )
        )
        self.mvit_blocks.append(
            MobileViTBlock(
                dims[2],
                L[2],
                conv_channels[9],
                self.kernel_size,
                patch_size,
                int(dims[2] * 4),
            )
        )

        self.final_pw = Conv2DBlock(
            in_channels=conv_channels[-2],
            out_channels=conv_channels[-1],
            kernel_size=1,
            stride=1,
            padding=0,
            groups=1,
            bias=False,
            norm=True,
            activation=False,
        )

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(conv_channels[-1], num_classes, bias=False)

        ## Training-related members
        self.criterion = nn.CTCLoss(blank=0, zero_infinity=True)
        self.wer = WordErrorRate()

        self.vocabulary = vocabulary

        self.apply(self.init_weights)  # Initialize weights

    def init_weights(self, m):

        if type(m) == nn.Conv2d:
            nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif type(m) == nn.BatchNorm2d:
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)
        elif type(m) == nn.Linear:
            nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

    def forward(self, x):

        B, T, C, H, W = x.shape
        x = rearrange(x, "b t c h w -> (b t) c h w")

        x = self.in_conv(x)

        for i in range(5):
            x = self.mv2_blocks[i](x)

        x = rearrange(x, "(b t) c h w -> b t c h w", b=B, t=T)
        x = self.mvit_blocks[0](x)

        x = rearrange(x, "b t c h w -> (b t) c h w")
        x = self.mv2_blocks[5](x)

        x = rearrange(x, "(b t) c h w -> b t c h w", b=B, t=T)
        x = self.mvit_blocks[1](x)

        x = rearrange(x, "b t c h w -> (b t) c h w")
        x = self.mv2_blocks[6](x)

        x = rearrange(x, "(b t) c h w -> b t c h w", b=B, t=T)
        x = self.mvit_blocks[2](x)

        x = rearrange(x, "b t c h w -> (b t) c h w", b=B, t=T)
        x = self.final_pw(x)
        x = self.pool(x)
        x = x.flatten(1)
        x = self.classifier(x)
        x = rearrange(x, "(b t) p -> t b p", b=B, t=T)  # time major due to CTC loss
        return x

    def step(self, batch, phase):
        x, y = batch

        B, T, C, H, W = x.shape
        B, N = y.shape

        # assume padding is done in the dataloader
        input_lengths = torch.full((B,), T, dtype=torch.long)
        target_lengths = torch.full((B,), N, dtype=torch.long)

        logits = self(x)

        log_probs = torch.nn.functional.log_softmax(logits, dim=-1)

        print(log_probs.shape, y.shape, input_lengths.shape, target_lengths.shape)
        print(input_lengths, target_lengths)
        loss = self.criterion(log_probs, y, input_lengths, target_lengths)

        self.log(f"{phase}/loss", loss, prog_bar=True)

        if phase != "train":
            logits = rearrange(logits, "t b p -> b t p")  # batch major for WER
            word_error_rate = self.calculate_wer(logits, y)
            self.log(f"{phase}/wer", word_error_rate, prog_bar=True)

        return loss

    def calculate_wer(self, logits, y):
        pred_strings = self.vocabulary.string_from_logits(logits)
        target_strings = self.vocabulary.string_from_ground_truth(y)

        return self.wer(pred_strings, target_strings)

    def training_step(self, batch, batch_idx):
        return self.step(batch, "train")

    def validation_step(self, batch, batch_idx):
        return self.step(batch, "val")

    def test_step(self, batch, batch_idx):
        return self.step(batch, "test")

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(), lr=2e-4, amsgrad=True, weight_decay=0.01
        )
        scheduler = {
            "scheduler": torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
                optimizer, T_0=10, T_mult=2, eta_min=1e-6
            ),
            "interval": "step",
        }

        return [optimizer], [scheduler]

In [11]:
class CustomVocab:

    def __init__(self, vocab, debug_mode=False):
        self.vocab = vocab
        self.debug_mode = debug_mode

    def __getitem__(self, idx):
        if idx >= len(self.vocab):
            if self.debug_mode:
                print("Index out of bounds, returning blank token")
            idx = 0
        return self.vocab[idx]

    def __call__(self, labels):
        # assume 0 is the blank token

        if type(labels) == torch.Tensor:
            return [self.vocab[(i).item()] for i in labels]
        else:
            return [self.vocab[(i)] for i in labels]

    def __len__(self):
        return len(self.vocab)

    def decode_ctc(self, predicted_tokens, blank=0):

        decoded = []
        for seq in predicted_tokens:  # Iterate over batch
            result = []
            prev_token = blank
            for token in seq:
                if token != prev_token and token != blank:
                    result.append(token.item() - 1)
                prev_token = token
            decoded.append(result)
        return decoded

    def string_from_logits(self, logits):

        pred_tkn = torch.argmax(logits, dim=-1)
        decoded = self.decode_ctc(pred_tkn)

        strings = []
        for b in range(len(decoded)):
            string = " ".join(self(decoded[b]))
            strings.append(string)

        return strings

    def string_from_ground_truth(self, labels):
        labels = labels - 1
        strings = []
        for b in range(len(labels)):
            string = " ".join(self(labels[b]))
            strings.append(string)

        return strings

In [36]:
curr_dir = os.getcwd()
datapath = "data/phoenix-2014.v3/phoenix2014-release/phoenix-2014-multisigner"
path = os.path.join(curr_dir, datapath)

dtrain = PhoenixVideoTextDataset(
    root=path,
    split="train",
    p_drop=0.2,
    random_drop=True,
)


dval = PhoenixVideoTextDataset(
    root=path,
    split="dev",
    p_drop=0.2,
    random_drop=True,
)

dtest = PhoenixVideoTextDataset(
    root=path,
    split="test",
    p_drop=0.2,
    random_drop=True,
)


def collate_add_blank(base_collate, batch):

    def resize_tensor(x):
        transform = torchvision.transforms.Resize((256, 192))
        B, T, C, H, W = x.shape
        x = rearrange(
            transform(rearrange(x, "b t c h w -> (b t) c h w")),
            "(b t) c h w -> b t c h w",
            b=B,
            t=T,
        )
        return x

    batch = base_collate(batch)  # add one to all labels
    batch["text"] = [tens + 1 for tens in batch["text"]]
    batch = (
        torch.stack(batch["video"], 0),
        torch.stack(batch["text"], 0),
    )

    batch = (resize_tensor(batch[0]), batch[1])

    return batch


# extend the collate function so that targets are tensorized (no padding is done, we use batch size 1 and gradient accumulation with the trainer)
collation = lambda x: collate_add_blank(dtrain.collate_fn, x)
dl_train = DataLoader(
    dtrain,
    collate_fn=collation,
    shuffle=True,
    batch_size=1,
    num_workers=3,
    pin_memory=True,
)
dl_val = DataLoader(
    dval,
    collate_fn=collation,
    shuffle=False,
    batch_size=1,
    num_workers=3,
    pin_memory=True,
)
dl_test = DataLoader(
    dtest,
    collate_fn=collation,
    shuffle=False,
    batch_size=1,
    num_workers=3,
    pin_memory=True,
)

# for some reason the MEIN token is not in the vocabulary, this speaks volumes about the quality of the dataset
dtrain.vocab.table["MEIN"] = 1231
# weirdly the dataset maps strings to labels, but we really want to map labels to strings
inv_table = {v: k for k, v in dtrain.vocab.table.items()}

vocab = CustomVocab(inv_table)

In [37]:
from lightning.pytorch.utilities.model_summary import summarize
from lightning.fabric.utilities import measure_flops

In [38]:
def xxs_mvit(vocab):
    return MobileViT(
        dims=[64, 80, 96],
        conv_channels=[16, 16, 24, 24, 48, 48, 64, 64, 80, 80, 320],
        num_classes=len(vocab) + 1,  # +1 for the blank token
        vocabulary=vocab,
        expand_ratio=2,
        patch_size=(2, 2),
    )


def xs_mvit(vocab):
    return MobileViT(
        dims=[96, 120, 144],
        conv_channels=[16, 32, 48, 48, 64, 64, 80, 80, 96, 96, 384],
        num_classes=len(vocab) + 1,  # +1 for the blank token
        vocabulary=vocab,
        expand_ratio=4,
        patch_size=(4, 4),
    )


def s_mvit(vocab):
    return MobileViT(
        dims=[144, 192, 240],
        conv_channels=[16, 32, 64, 64, 96, 96, 128, 128, 160, 160, 640],
        num_classes=len(vocab) + 1,  # +1 for the blank token
        vocabulary=vocab,
        expand_ratio=4,
        patch_size=(8, 8),
    )

In [39]:
import wandb

# UNCOMMENT THIS TO LOG TO WANDB
# from kaggle_secrets import UserSecretsClient
#
# user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("wandb_api")
#
# wandb.login(key=secret_value_0)

In [40]:
# wandb.init(project="mobilevit")

In [41]:
batch = next(iter(dl_val))

In [42]:
X, y = batch

In [43]:
y - 1

tensor([[ 417,  715, 1061, 1115,  561,  921,  898, 1029, 1231,  601]])

In [44]:
model = xxs_mvit(vocab)

In [45]:
model.vocabulary.string_from_ground_truth(y)

['ICH OSTERN WETTER ZUFRIEDEN MITTAG TEMPERATUR SUED WARM MEIN NICHT']

In [46]:
import lightning.pytorch as lp

# setup training and wandb

BATCH_SIZE = 1

wandb_logger = lp.loggers.WandbLogger()

model = xxs_mvit(vocab)


trainer = pl.Trainer(
    max_epochs=10,
    accelerator="cpu",
    logger=wandb_logger,
    callbacks=[
        lp.callbacks.ModelCheckpoint(
            monitor="val/loss",
            filename="best_model",
            save_top_k=1,
            mode="min",
        ),
        lp.callbacks.EarlyStopping(
            monitor="val/ loss",
            patience=3,
            mode="min",
        ),
    ],
    accumulate_grad_batches=16,
)

wandb_logger.watch(model, log_graph=False)

trainer.fit(model, dl_train, dl_val)

wandb.finish()

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: loi-1940849 (academic_projects). Use `wandb login --relogin` to force relogin



  | Name        | Type              | Params | Mode 
----------------------------------------------------------
0 | in_conv     | Conv2DBlock       | 464    | train
1 | mv2_blocks  | ModuleList        | 45.7 K | train
2 | mvit_blocks | ModuleList        | 1.1 M  | train
3 | final_pw    | Conv2DBlock       | 26.2 K | train
4 | pool        | AdaptiveAvgPool2d | 0      | train
5 | classifier  | Linear            | 394 K  | train
6 | criterion   | CTCLoss           | 0      | train
7 | wer         | WordErrorRate     | 0      | train
----------------------------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.241     Total estimated model params size (MB)
292       Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]torch.Size([172, 1, 1233]) torch.Size([1, 10]) torch.Size([1]) torch.Size([1])
tensor([172]) tensor([10])
Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:19<00:19,  0.05it/s]torch.Size([144, 1, 1233]) torch.Size([1, 19]) torch.Size([1]) torch.Size([1])
tensor([144]) tensor([19])



Detected KeyboardInterrupt, attempting graceful shutdown ...


RuntimeError: Please call `iter(combined_loader)` first.

In [ ]:
# flush cuda cache
torch.cuda.empty_cache()


wandb.finish()

In [ ]:
from phoenix_datasets import PhoenixVideoTextDataset
import torch
from torch.utils.data import DataLoader

import os

curr_dir = os.getcwd()
datapath = "data/phoenix-2014.v3/phoenix2014-release/phoenix-2014-multisigner"
path = os.path.join(curr_dir, datapath)

dtrain = PhoenixVideoTextDataset(
    # your path to this folder, download it from official website first.
    root=path,
    split="train",
    p_drop=0.5,
    random_drop=True,
)


def collate_add_blank(base_collate, batch):

    batch = base_collate(batch)  # add one to all labels
    batch["text"] = [tens + 1 for tens in batch["text"]]
    batch = (
        torch.stack(batch["video"], 0),
        torch.stack(batch["text"], 0),
    )

    return batch


vocab = dtrain.vocab

print("Vocab", vocab)

collation = lambda x: collate_add_blank(dtrain.collate_fn, x)

dl = DataLoader(dtrain, collate_fn=collation, shuffle=True, batch_size=1)

for batch in dl:
    video, label = batch

    print(video.shape, label.shape)

    break

In [ ]:
vocab_inv = {(v + 1): k for k, v in vocab.table.items()}
len(vocab_inv)

In [338]:
CV = CustomVocab(vocab_inv)

In [ ]:
" ".join(labels_to_text(label[0], vocab_inv))

In [ ]:
label[0]

In [317]:
# get logits with 1.0 on each label for the first batch
logits = torch.zeros(2, len(label[0]), len(vocab_inv) + 1)

In [ ]:
vocab_inv[399]

In [323]:
# set logits to 1.0 for each label
logits[:, torch.arange(len(label[0])), label[0]] = 1.0

In [ ]:
CV.string_from_logits(logits)

In [ ]:
torch.argmax(logits, dim=-1)

In [ ]:
" ".join(labels_to_text(torch.argmax(logits, dim=-1)[0], vocab_inv))

In [246]:
from pyctcdecode import build_ctcdecoder

In [247]:
labels = list(CV.vocab.values())

In [ ]:
decoder = build_ctcdecoder(labels)

In [ ]:
len(labels)

In [ ]:
decoder.decode_beams(logits.squeeze(0).numpy())